In [1]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Dropout,Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import display
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.applications import VGG19
from keras.constraints import maxnorm
from keras import optimizers
from keras.optimizers import Adam, SGD , RMSprop
import scipy.misc
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from PIL import Image

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.
W0728 14:26:55.642529 140513486710656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:153: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



In [0]:
num_px = 128
C = 2
B_size = 64
E = 10

In [3]:
base_model = VGG19(weights = "imagenet", include_top=False, 
                   input_shape = (num_px, num_px, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(0.4)(x)
x = Dense(256, activation = "relu")(x)
x = Dropout(0.2)(x)
preds = Dense(C, activation = "softmax")(x)

model = Model(input = base_model.input, output = preds)

W0728 14:26:56.193850 140513486710656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 14:26:56.204367 140513486710656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 14:26:56.235605 140513486710656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0728 14:26:56.672104 140513486710656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0728 14:26:56.673324 140513486710656 deprecation_

In [0]:
for layer in model.layers[:16]:
    layer.trainable=False
for layer in model.layers[16:]:
    layer.trainable=True

In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [0]:
early = EarlyStopping(monitor = 'val_acc', min_delta = 0, 
                      patience = 10, verbose = 1 , mode = 'auto')

In [7]:
model.compile(loss = "categorical_crossentropy", 
              optimizer = SGD(lr=0.001, momentum=0.9), 
              metrics=["accuracy"])

W0728 14:26:57.799508 140513486710656 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [8]:
trainAug = ImageDataGenerator(rescale = 1./255, fill_mode = "nearest")

trainGen = trainAug.flow_from_directory(
                    '/content/chest_xray/train',
                    target_size = (num_px, num_px),
                    batch_size = B_size,
                    shuffle = True,
                    class_mode = "categorical")

Found 5216 images belonging to 2 classes.


In [9]:
testGen = trainAug.flow_from_directory(
                    '/content/chest_xray/test',
                    target_size = (num_px, num_px),
                    batch_size = B_size,
                    shuffle = False,
                    class_mode = "categorical")

Found 624 images belonging to 2 classes.


In [10]:
ValGen = trainAug.flow_from_directory(
                    '/content/chest_xray/val',
                    target_size = (num_px, num_px),
                    batch_size = B_size,
                    shuffle = False,
                    class_mode = "categorical")

Found 16 images belonging to 2 classes.


In [11]:
H = model.fit_generator(trainGen, epochs=E, validation_data = ValGen, validation_steps = 16/64,steps_per_epoch = 5216//64, callbacks = [early])

W0728 14:27:43.549371 140513486710656 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
81/81 [==============================] - 55s 675ms/step - loss: 0.2092 - acc: 0.9130 - val_loss: 0.3780 - val_acc: 0.8750
Epoch 2/10
81/81 [==============================] - 50s 619ms/step - loss: 0.1155 - acc: 0.9562 - val_loss: 0.4374 - val_acc: 0.8750
Epoch 3/10
81/81 [==============================] - 50s 616ms/step - loss: 0.0644 - acc: 0.9769 - val_loss: 0.5142 - val_acc: 0.8125
Epoch 4/10
81/81 [==============================] - 50s 614ms/step - loss: 0.0679 - acc: 0.9749 - val_loss: 0.5431 - val_acc: 0.6875
Epoch 5/10
81/81 [==============================] - 49s 611ms/step - loss: 0.0402 - acc: 0.9844 - val_loss: 0.3356 - val_acc: 0.8125
Epoch 6/10
81/81 [==============================] - 50s 619ms/step - loss: 0.0426 - acc: 0.9850 - val_loss: 1.2905 - val_acc: 0.6875
Epoch 7/10
81/81 [==============================] - 49s 608ms/step - loss: 0.0320 - acc: 0.9886 - val_loss: 0.2235 - val_acc: 0.9375
Epoch 8/10
81/81 [==============================] - 50s 611ms/step - 

In [0]:
model.save('model.h5')

In [13]:
testGen.reset()
predicts = model.predict_generator(testGen, steps = (624 // B_size) + 1) 
predicts = np.argmax(predicts, axis=1)

print("Test set accuracy: " + 
      str(accuracy_score(testGen.classes, predicts, normalize=True) * 100) 
      + "%") 

Test set accuracy: 76.92307692307693%
